In [83]:
# Importing the important libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [84]:
#Load Dataset
df = pd.read_csv(r"C:\Users\Pranita\OneDrive\Desktop\VE3_ML_Deply\House_Price_Data.csv")

In [85]:
df.head()  #Read the data

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [86]:
df.sample(5)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
3793,Plot Area,Ready To Move,Geetanjali Layout,4 Bedroom,NaN,1200,4.0,2.0,300.0
13049,Super built-up Area,Ready To Move,Kadugodi,2 BHK,NaN,1196,2.0,3.0,60.0
2771,Built-up Area,Ready To Move,Bommanahalli,2 BHK,NaN,1090,3.0,2.0,44.0
9907,Plot Area,Ready To Move,Pampa Extension,4 Bedroom,NaN,1200,4.0,2.0,250.0
4206,Super built-up Area,Ready To Move,Tumkur Road,3 BHK,SosonGa,1500,3.0,3.0,100.0


In [87]:
# View data types and non-null counts
df.info

<bound method DataFrame.info of                   area_type   availability                  location  \
0      Super built-up  Area         19-Dec  Electronic City Phase II   
1                Plot  Area  Ready To Move          Chikka Tirupathi   
2            Built-up  Area  Ready To Move               Uttarahalli   
3      Super built-up  Area  Ready To Move        Lingadheeranahalli   
4      Super built-up  Area  Ready To Move                  Kothanur   
...                     ...            ...                       ...   
13315        Built-up  Area  Ready To Move                Whitefield   
13316  Super built-up  Area  Ready To Move             Richards Town   
13317        Built-up  Area  Ready To Move     Raja Rajeshwari Nagar   
13318  Super built-up  Area         18-Jun           Padmanabhanagar   
13319  Super built-up  Area  Ready To Move              Doddathoguru   

            size  society total_sqft  bath  balcony   price  
0          2 BHK  Coomee        1056   2.

In [88]:
print(df.shape)
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns')

(13320, 9)
The dataset has 13320 rows and 9 columns


In [89]:
# Dropping the redundant features
df = df.drop(['availability','society', 'area_type','balcony'], axis=1)
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [90]:
# Handling Missing Values
df.isnull().sum().sort_values(ascending=False)


bath          73
size          16
location       1
total_sqft     0
price          0
dtype: int64

In [91]:
df = df.dropna()

In [92]:
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      shape=(2067,), dtype=object)

In [93]:
#Clean total_sqft Column
def is_float(x):
    if x is None or x == '':
        return False
    try:
        float(x)
    except:
        return False
    return True

print(len(df[~df['total_sqft'].apply(is_float)]))
## Filter non-float values
df = df[df['total_sqft'].apply(is_float)]
df['total_sqft'] = df['total_sqft'].astype('float')

190


In [94]:
# Add BHK Column & Drop Size Col
df['bhk'] = df['size'].apply(lambda x: int(x.split(' ')[0]))
df.head(2)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4


In [95]:
df = df.drop('size', axis=1)

In [96]:
#Simplify Location Data
df['location'].value_counts().head(10)

location
Whitefield               517
Sarjapur  Road           386
Electronic City          302
Kanakpura Road           260
Thanisandra              222
Yelahanka                203
Uttarahalli              185
Marathahalli             175
Hebbal                   172
Raja Rajeshwari Nagar    171
Name: count, dtype: int64

In [97]:
loc = df['location'].value_counts()
print(len(loc[loc>10]))
print(len(loc[loc<=10]))
loc_less_10 = loc[loc <= 10].index.tolist()
df['location'] = df['location'].apply(lambda x: 'other' if x in loc_less_10 else x)
df['location'].nunique()

239
1059


240

In [98]:
df[df['total_sqft']/df['bhk']<300].head()

,location,total_sqft,bath,price,bhk
9,other,1020.0,6.0,370.0,6
45,HSR Layout,600.0,9.0,200.0,8
58,Murugeshpalya,1407.0,4.0,150.0,6
68,other,1350.0,7.0,85.0,8
70,other,500.0,3.0,100.0,3


In [99]:
df= df[~(df['total_sqft']/df['bhk']<300)]

In [100]:
# Label Encode Location
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['location_encoded'] = le.fit_transform(df['location'])
print(df[['location', 'location_encoded']].head(3))

                   location  location_encoded
0  Electronic City Phase II                79
1          Chikka Tirupathi                61
2               Uttarahalli               224


In [101]:
df.head(5)

,location,total_sqft,bath,price,bhk,location_encoded
0,Electronic City Phase II,1056.0,2.0,39.07,2,79
1,Chikka Tirupathi,2600.0,5.0,120.00,4,61
2,Uttarahalli,1440.0,2.0,62.00,3,224
3,Lingadheeranahalli,1521.0,3.0,95.00,3,158
4,Kothanur,1200.0,2.0,51.00,2,150


In [102]:
df = df.drop('location',axis=1)
df.head(3)

,total_sqft,bath,price,bhk,location_encoded
0,1056.0,2.0,39.07,2,79
1,2600.0,5.0,120.00,4,61
2,1440.0,2.0,62.00,3,224


In [103]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso


In [104]:
X = df.drop(['price'], axis=1)
y = df['price']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [105]:
#Train Linear Regression Model
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.2214084516620476

In [106]:
# Cross Validation

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.3392404 , 0.41315532, 0.4027344 , 0.37841465, 0.47811749])

In [107]:
#Grid Search for Best Model

from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

def find_best_model_using_gridsearchcv(X, y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['squared_error', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }

    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X, y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

result = find_best_model_using_gridsearchcv(X, y)
result

,model,best_score,best_params
0,linear_regression,0.402332,{'fit_intercept': True}
1,lasso,0.403273,"{'alpha': 2, 'selection': 'cyclic'}"
2,decision_tree,0.338054,"{'criterion': 'friedman_mse', 'splitter': 'ran..."


In [108]:
#Prediction Func


def predict_price(location,sqft,bath,bhk):
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

In [109]:
# Save the Model
import pickle
model_columns = X.columns.tolist()

with open('house_price_model.pkl','wb') as f:
    pickle.dump({'model': lr_clf, 'columns': model_columns}, f)
